In [1]:
import torch

def corr2d_multi_in(x, k):
    return sum(corr2d(x, k) for x, k in zip(x, k))

In [2]:
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = reduce_sum((X[i:i + h, j:j + w] * K))
    return Y

reduce_sum = lambda x, *args, **kwargs: x.sum(*args, **kwargs)

In [3]:
x = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
                  [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
k = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(x, k)

tensor([[ 56.,  72.],
        [104., 120.]])

In [4]:
def corr2d_multi_in_out(x, k):
    return torch.stack([corr2d_multi_in(x, k) for k in k], 0)

In [5]:
k = torch.stack((k, k + 1, k + 2), 0)
k.shape

torch.Size([3, 2, 2, 2])

In [6]:
corr2d_multi_in_out(x, k)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [7]:
def corr2d_multi_in_out_1x1(x, k):
    c_i, h, w = x.shape
    c_o = k.shape[0]
    x = x.reshape((c_i, h * w))
    k = k.reshape((c_o, c_i))
    y = torch.matmul(k, x)
    return y.reshape((c_o, h, w))

In [8]:
x = torch.normal(0, 1, (3, 3, 3))
k = torch.normal(0, 1, (2, 3, 1, 1))

y1 = corr2d_multi_in_out_1x1(x, k)
y2 = corr2d_multi_in_out(x, k)
assert float(torch.abs(y1 - y2).sum()) < 1e-6